In [1]:
import pandas as pd
import json
import requests
from functools import partial

pd.set_option("display.max_columns", 100)

In [11]:
def get_trade_df(page):
    # make a team map constant based on the js code from the webiste
    # found in this file on line 91
    # https://transactions.mlbtraderumors.com/widget/transactions-tracker&lang=en
    
    TEAM_ABBR_MAP = {}
    TEAM_ABBR_MAP[32]="ANA";
    TEAM_ABBR_MAP[1]="ARZ";
    TEAM_ABBR_MAP[2]="ATL";
    TEAM_ABBR_MAP[3]="BAL";
    TEAM_ABBR_MAP[4]="BOS";
    TEAM_ABBR_MAP[5]="CHC";
    TEAM_ABBR_MAP[6]="CWS";
    TEAM_ABBR_MAP[7]="CIN";
    TEAM_ABBR_MAP[8]="CLE";
    TEAM_ABBR_MAP[9]="COL";
    TEAM_ABBR_MAP[10]="DET";
    TEAM_ABBR_MAP[11]="FLA";
    TEAM_ABBR_MAP[12]="HOU";
    TEAM_ABBR_MAP[13]="KAN";
    TEAM_ABBR_MAP[14]="LAA";
    TEAM_ABBR_MAP[15]="LAD";
    TEAM_ABBR_MAP[31]="MIA";
    TEAM_ABBR_MAP[16]="MIL";
    TEAM_ABBR_MAP[17]="MIN";
    TEAM_ABBR_MAP[18]="NYM";
    TEAM_ABBR_MAP[19]="NYY";
    TEAM_ABBR_MAP[20]="OAK";
    TEAM_ABBR_MAP[21]="PHI";
    TEAM_ABBR_MAP[22]="PIT";
    TEAM_ABBR_MAP[23]="SDG";
    TEAM_ABBR_MAP[24]="SFO";
    TEAM_ABBR_MAP[25]="SEA";
    TEAM_ABBR_MAP[26]="STL";
    TEAM_ABBR_MAP[33]="TBR";
    TEAM_ABBR_MAP[27]="TAM";
    TEAM_ABBR_MAP[28]="TEX";
    TEAM_ABBR_MAP[29]="TOR";
    TEAM_ABBR_MAP[30]="WAS";
    TEAM_ABBR_MAP[20000]="JAP";

    
    # each requests gets 75 results at a time, use page to paginate
    
    json_response = requests.get(f"https://transactions.mlbtraderumors.com/g/GetTransactions&widget=true&amount_type=4&type_ID=1&lang=41&OFFSET={page * 75}").json()
    df = pd.DataFrame(json_response["transactions"])
    
    # make numbers numeric and ignore errors
    df = df.apply(partial(pd.to_numeric, errors="ignore"))
    
    # map team names
    df["from"] = df.FORMER_TEAM.dropna().astype(int).map(TEAM_ABBR_MAP)
    df["to"] = df.ACQUIRING_TEAM.dropna().astype(int).map(TEAM_ABBR_MAP)
    
    # return the df for the page
    return df
    

In [12]:
get_trade_df(1).head(2) # quick test

,TRANSACTION_ID,TRANSACTION_TYPE,TRANSACTION_TYPE_ID,TRANSACTION_DATE,C_TRANSACTION_DATE,FREE_AGENT_TYPE_ID,QUALIFYING_OFFER_TYPE_ID,PLAYER_ID,PLAYER_NAME,FORMER_GM,ACQUIRING_GM,FORMER_TEAM,ACQUIRING_TEAM,AGENCY_ID,CONTRACT_YEARS,CONTRACT_AMOUNT,CONTRACT_BUYOUT,MLBTR_LINK,BBREF_TRAN_ID,NOTES,DATE_CREATED,STAFF_CREATED,DATE_UPDATED,STAFF_UPDATED,DATE_DISABLED,from,to
0,63252,Trade,1,2019-08-10 00:00:00,08/10/2019,NaN,NaN,22384,Ian Miller,34,121,25,17,NaN,NaN,NaN,NaN,https://www.mlbtraderumors.com/2019/08/twins-t...,NaN,Twins acquire Ian Miller from Mariners in exch...,2019-08-11 14:14:22,28,"August 11, 2019, 2:14PM",JD Shaw,NaN,SEA,MIN
1,63245,Trade,1,2019-08-09 00:00:00,08/09/2019,NaN,NaN,9269,Jose Lobaton,34,2,25,15,209.0,NaN,NaN,NaN,https://www.mlbtraderumors.com/2019/08/dodgers...,NaN,Dodgers acquire Jose Lobaton from Mariners in ...,2019-08-10 17:06:08,28,"August 10, 2019, 5:06PM",JD Shaw,NaN,SEA,LAD


In [13]:
%%time
# get 10 pages worth of data
df = pd.concat( [ get_trade_df(page) for page in range(10)])

CPU times: user 296 ms, sys: 0 ns, total: 296 ms
Wall time: 7.06 s


In [14]:
# check the date range
print("from", pd.to_datetime(df.TRANSACTION_DATE).min(), "to", pd.to_datetime(df.TRANSACTION_DATE).max())

from 2018-01-17 00:00:00 to 2020-01-17 00:00:00


In [15]:
# get data for 2019 and on
df_2019 = df[pd.to_datetime(df.TRANSACTION_DATE) >= '01-01-2019']

In [16]:
# export summary to csv
df_2019.groupby(["from", "to"]).size().reset_index().to_csv("players_traded_teams.csv", index=False)

In [17]:
# export all data to csv
df.to_csv("all_trade_data.csv", index=False)